In [1]:
import os
import random
import numpy as np
import tensorflow as tf
seed = 1
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
import gensim
import pandas as pd
import gensim.downloader as api
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D

In [4]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [5]:
def convert_to_sequence(questions):
    X = questions['Preprocessed_Question']
    cognitive_level = {"Knowledge": 0, "Comprehension": 1, "Application": 2, "Analysis": 3, "Synthesis": 4, "Evaluation": 5}
    questions["BT LEVEL"].replace(cognitive_level, inplace = True)
    y = questions['BT LEVEL'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.10, 
                                        stratify = questions['BT LEVEL'], random_state = 1)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    tokenizer = Tokenizer(oov_token ='<OOV>')
    tokenizer.fit_on_texts(X_train)
    vocab = tokenizer.word_index
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    max_que_len = max(len(x) for x in X_train_seq)
    padded_X_train_seq = pad_sequences(X_train_seq, maxlen = max_que_len, padding = 'post')
    padded_X_test_seq = pad_sequences(X_test_seq, maxlen = max_que_len, padding = 'post')
    
    return padded_X_train_seq, y_train, padded_X_test_seq, y_test, vocab, max_que_len

In [6]:
def get_embedding(vocab):
    embedding_model = api.load('glove-wiki-gigaword-300')
    embeddings = np.zeros((len(vocab) + 1, 300))
    for word, index in vocab.items():
        if word in embedding_model.key_to_index:
            embedding_vector = embedding_model[word]
            embeddings[index] = embedding_vector
        else:
            print(word)
    return embeddings

In [7]:
def build_model(vocab_size, embedding_len, max_que_len, embeddings):
    # defining the model
    model = Sequential()
    model.add(Embedding(input_dim = vocab_size, output_dim = embedding_len, weights = [embeddings], 
                    input_length = max_que_len,  trainable = True))
    model.add(Conv1D(32, 5, activation ='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(6, activation ='softmax'))
    return model

In [21]:
def evaluate_model(model, X_test, y_test):
    pred = model.predict(X_test)
    predicted_classes = np.argmax(pred, axis = 1)
    true_classes = np.argmax(y_test, axis = 1)
    accuracy = accuracy_score(true_classes, predicted_classes)
    print(f"Accuracy: {accuracy}")
    f1_ = f1_score(true_classes, predicted_classes, average = 'weighted')
    print(f"F1 score: {f1_}")
    print(classification_report(true_classes, predicted_classes))

In [9]:
def plot_result(log_data): 
    plt.figure(figsize=(10, 6))
    plt.plot(log_data['loss'], label ='train')
    plt.plot(log_data['val_loss'], label ='test')
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.figure(figsize=(10, 6))
    plt.plot(log_data['accuracy'], label ='train')
    plt.plot(log_data['val_accuracy'], label ='test')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    plt.plot(log_data['f1_score'], label ='train')
    plt.plot(log_data['val_f1_score'], label ='test')
    plt.title('Model F1 Score')
    plt.ylabel('F1 Score')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

In [10]:
questions = pd.read_excel('preprocessing_result/preprocessing_result-glove-wiki.xlsx')
questions

,Question,Preprocessed_Question,BT LEVEL
0,"Suppose prices of two goods are constant, expl...",suppose price good constant explain what happe...,Comprehension
1,Explain the concept of price leadership observ...,explain concept price leadership observe condi...,Comprehension
2,Define profit. Briefly explain how accounting ...,define profit briefly explain how account prof...,Comprehension
3,Describe the assumptions of monopolistic compe...,describe assumption monopolistic competitive m...,Comprehension
4,Explain the meaning of the law of diminishing ...,explain mean law diminish marginal return brie...,Comprehension
...,...,...,...
2517,PEST and SWOT are popular strategy tools. Disc...,pest swot popular strategy tool discuss how to...,Application
2518,List the advantages and disadvantages of Publi...,list advantage disadvantage public offer,Knowledge
2519,Show your calculations for all THREE (3) optio...,show calculation option discuss which option p...,Analysis
2520,Currently the product life cycle for Apple iPo...,currently product life cycle apple ipod growth...,Evaluation


In [11]:
padded_X_train_seq, y_train, padded_X_test_seq, y_test, vocab, max_que_len = convert_to_sequence(questions)

In [12]:
# embeddings_glove_wiki = get_embedding(vocab)
# np.save('embedding/glove/embeddings.npy', embeddings_glove_wiki)

<OOV>
aov
infomediary
phosphoru
mccg
komugi
eoq
elute
finfet
utar
sqrt
chemostat
radiobutton
adts
kaseem
perakian
junebank
firedup
qlassic
mirr
technopreneur
pondy
chemisorption
quasineutral
openerp
spiration
electroosmosis
hexapeptide
kmk
nextseq
passivator
arraylist
bubblesort
matrik
putable
putlog
blosum
monopolistically
dictogloss
hyperarid
eqrnhl
neolocal
sethour
sendredirect
linebreeding
phenylpropanoid
pourbaix
minisatellite
oxyperoxidase
luedeking
pullulan
struc
mesofauna
cominunicate
medoids
diskretionary
langmiur
isoterm
requestdispatcher
mircroscope
higg
httpsessionactivationlistener
mincut
determinental
schubli
mediuin
sqykrecp
myner
programrs
wmeadk
dmvrel
mmlr
allelopathy
luzonensis
ionizable
prenyltransferase
exarnple
blastx
pnk
equillibrium
musictype
mytune
fupo
placentome
transabdominal
matsuhara
deae
sephadex
jstl
outbred
elaeis
eisenthal
modelessform
showdialog
herbatious
jfet
makamat
incfromn
sooosay
outli
ytm
komive
tblastx
viestal
atcase
communalitie
miscroscope


In [13]:
embeddings_glove_wiki = np.load('embedding/glove/embeddings.npy')

In [14]:
#callbacks 
training_logger = CSVLogger('log/Glove/training_wiki.log', separator = ',', append = False)

In [15]:
#model
model = build_model(len(vocab) + 1, 300, max_que_len, embeddings_glove_wiki)
# compiling the model
model.compile(optimizer = 'RMSprop', loss='categorical_crossentropy', metrics=['accuracy', tfa.metrics.F1Score(6, 'weighted')])
# printing summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 44, 300)           1114200   
                                                                 
 conv1d (Conv1D)             (None, 40, 32)            48032     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 1,162,430
Trainable params: 1,162,430
Non-trainable params: 0
_________________________________________________________________


In [16]:
# fit the model
model.fit(padded_X_train_seq, y_train, epochs = 50, batch_size = 8, validation_data = (padded_X_test_seq, y_test),
                                                                      callbacks = [training_logger], verbose = 1)
#saving the trained model
model.save('saved_models/Glove/EQCM_wiki.h5')

Epoch 1/50
284/284 [==============================] - 4s 10ms/step - loss: 1.2790 - accuracy: 0.5381 - f1_score: 0.5095 - val_loss: 0.8616 - val_accuracy: 0.7391 - val_f1_score: 0.7321
Epoch 2/50
284/284 [==============================] - 3s 9ms/step - loss: 0.5306 - accuracy: 0.8475 - f1_score: 0.8463 - val_loss: 0.7353 - val_accuracy: 0.7787 - val_f1_score: 0.7748
Epoch 3/50
284/284 [==============================] - 2s 8ms/step - loss: 0.2300 - accuracy: 0.9493 - f1_score: 0.9493 - val_loss: 0.6798 - val_accuracy: 0.7905 - val_f1_score: 0.7855
Epoch 4/50
284/284 [==============================] - 2s 8ms/step - loss: 0.0873 - accuracy: 0.9872 - f1_score: 0.9872 - val_loss: 0.6756 - val_accuracy: 0.7628 - val_f1_score: 0.7629
Epoch 5/50
284/284 [==============================] - 2s 8ms/step - loss: 0.0354 - accuracy: 0.9907 - f1_score: 0.9907 - val_loss: 0.7785 - val_accuracy: 0.7708 - val_f1_score: 0.7675
Epoch 6/50
284/284 [==============================] - 2s 8ms/step - loss: 0.012

In [17]:
log_data = pd.read_csv('log/Glove/training_wiki.log', sep = ',', engine = 'python')
log_data.head(50)

,epoch,accuracy,f1_score,loss,val_accuracy,val_f1_score,val_loss
0,0,0.538123,0.509456,1.278968e+00,0.739130,0.732118,0.861599
1,1,0.847510,0.846347,5.305675e-01,0.778656,0.774834,0.735253
2,2,0.949317,0.949271,2.299964e-01,0.790514,0.785471,0.679846
3,3,0.987219,0.987248,8.729480e-02,0.762846,0.762895,0.675591
4,4,0.990745,0.990739,3.536326e-02,0.770751,0.767454,0.778476
5,5,0.998237,0.998237,1.262203e-02,0.778656,0.774937,0.839458
6,6,0.999119,0.999118,4.558954e-03,0.766798,0.763349,1.000701
7,7,0.999119,0.999119,2.404380e-03,0.794466,0.792718,1.014408
8,8,1.000000,1.000000,6.034739e-04,0.790514,0.788600,1.037116
9,9,1.000000,1.000000,2.230115e-04,0.774704,0.769833,1.195944


In [18]:
log_data['val_accuracy'].describe()

count    50.000000
mean      0.781028
std       0.008526
min       0.739130
25%       0.778656
50%       0.782609
75%       0.786561
max       0.794466
Name: val_accuracy, dtype: float64

In [19]:
log_data['val_f1_score'].describe()

count    50.000000
mean      0.778422
std       0.009156
min       0.732118
25%       0.776492
50%       0.779179
75%       0.782733
max       0.792718
Name: val_f1_score, dtype: float64

In [22]:
model.load_weights("saved_models/Glove/EQCM_wiki.h5")
evaluate_model(model, padded_X_test_seq, y_test)

8/8 [==============================] - 0s 5ms/step
Accuracy: 0.7786561264822134
F1 score: 0.7783267267072054
              precision    recall  f1-score   support

           0       0.69      0.77      0.73        35
           1       0.83      0.85      0.84        96
           2       0.67      0.62      0.65        32
           3       0.86      0.80      0.83        30
           4       0.72      0.75      0.74        28
           5       0.82      0.72      0.77        32

    accuracy                           0.78       253
   macro avg       0.76      0.75      0.76       253
weighted avg       0.78      0.78      0.78       253



In [ ]:
log_data[['val_accuracy']].idxmax() + 1

In [ ]:
plot_result(log_data)